# fundamentals

In [1]:
"""
fundamentals exercise
"""
from IPython.core.display import display
from pyspark import dbutils as dbu
from requests import get
from zipfile import ZipFile
import os
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
dbutils = dbu.DBUtils(spark.sparkContext())

data_url = 'https://github.com/jtviegas/datasets/raw/main/fundamentals.zip'
zip_file_path = '/tmp/fundamentals.zip'
local_folder = '/tmp'
companies_file = 'companies_subset.csv'
indicators_file = 'indicators_subset.csv'
parquet_file = '/tmp/fundamentals/parquet'

local_companies = 'file:{}/{}'.format(local_folder,companies_file)
local_indicators = 'file:{}/{}'.format(local_folder,indicators_file)
dbfs_companies = 'dbfs:{}/{}'.format(local_folder,companies_file)
dbfs_indicators = 'dbfs:{}/{}'.format(local_folder,indicators_file)

with open(zip_file_path, "wb") as file:
  res = get(data_url)

  file.write(res.content)

with ZipFile(zip_file_path, 'r') as zipfile:
  files = zipfile.namelist()
  for file in files:
    zipfile.extract(file, local_folder)

os.listdir(local_folder)
dbutils.fs.mv(local_companies, dbfs_companies)
dbutils.fs.mv(local_indicators, dbfs_indicators)


dfc = spark.read.format('csv').option('inferSchema', 'true').option('header','true').load(dbfs_companies)
dfi = spark.read.format('csv').option('inferSchema', 'true').option('header','true').load(dbfs_indicators)
dbutils.fs.rm(parquet_file, recurse=True)
dfc.join(dfi, ['company_id'], 'left').write.parquet(parquet_file)

df = spark.read.parquet(parquet_file)
dbutils.fs.rm(dbfs_companies)
dbutils.fs.rm(dbfs_indicators)
dbutils.fs.rm(local_companies)
dbutils.fs.rm(local_indicators)
dbutils.fs.rm(zip_file_path)

print(spark.conf.get("spark.databricks.io.cache.enabled"))
display(df)




TypeError: 'SparkContext' object is not callable